In [1]:
import numpy as np
import random
import time
import os
import math
import statistics
from multiprocessing import Pool
import matplotlib.pyplot as plt


In [2]:
from csvreader import read_patient_csv

patientSet = read_patient_csv()
Adults = []
for patient in patientSet:
    if patient["age"] > 17:
        Adults.append(patient)

In [3]:
def create_new_set(setlength):
    new_set = []
    for i in range(setlength):
        new_set.append(round(random.random(),4))
    count = 0
    for k in new_set:
        i = round(k, 4)
        new_set[count] = i
        count += 1

        return new_set

In [4]:
from PyTCI.weights import leanbodymass
from PyTCI.models import propofol
from patient_solver import solve_for_patient
from csvreader import read_patient_csv


class New_model(propofol.Propofol):
    def __init__(self, age, weight, height, sex, params):
        
        lean_body_mass = leanbodymass.hume66(height, weight, sex)
        
        self.v1 = ((params[1] * 50) - params[2]*(age - (params[3] * 100))) * (params[4] * (lean_body_mass - (params[5] * 100)))
        self.v2 = params[6] * lean_body_mass * 2
        self.v3 = params[7] * weight * 5
        
        self.Q1 = ((params[8] * self.v1) * (params[9] * age)) ** 0.75
        self.Q2 = (params[9] * self.v2)** 0.75
        self.Q3 = (params[0] * self.v3) ** 0.75
        
        self.keo = 0
        
        propofol.Propofol.from_clearances(self)
        propofol.Propofol.setup(self)
        
class HackDay_model(propofol.Propofol):
    def __init__(self, age, weight, height, sex, params):
        
        lean_body_mass = leanbodymass.hume71(height, weight, sex)
        
        self.v1 = (params[1] * lean_body_mass) * (age **(-params[2]))
        self.v2 = params[3] * lean_body_mass * 2
        self.v3 = params[4] * weight * 5
        
        self.Q1 = (params[5] * (weight ** 0.75) ) * (age ** (-params[6]))
        self.Q2 = params[7] * (self.v2 ** 0.75)
        self.Q3 = params[8] * (self.v3 ** 0.75)
        
        self.keo = 0
        
        propofol.Propofol.from_clearances(self)
        propofol.Propofol.setup(self)        
                                                         
                   
                   
                   
def solve_for_custom(patient, params):
    patient_model = HackDay_model(patient["age"], patient["weight"], patient["height"], patient["sex"], params )
    return solve_for_patient(patient_model, patient["events"])


In [5]:
def create_new_population(size: int, setlength: int):
    pop_size = size
    pop_list = []
    for i in range(pop_size):
        newparam = create_new_set(setlength)
        pop_list.append(newparam)
    return pop_list

In [6]:
def mutate_population(children :int, fittest: list, second: list, mutants: int):
    pop_list = []
    
    #keep fittest set in so we don't go backwards
    pop_list.append(fittest)
    def mutate_chromosome(chrome):
        b = random.random()
        c = len(fittest)
        c = 1 / c
        if b < (c/2):
            chrome = chrome * np.random.normal(1, 0.3)
        elif b < c:
            chrome = chrome * np.random.normal(1, 0.1)
        else:
            chrome = chrome * 1
        return round(chrome, 6)

    # breed parents to create children
    def breed(sprogs, p1, p2):
        for i in range(sprogs):
            child = []
            count = 0
            for k in p1:
                a = random.random()
                if a < 0.5:
                    k = p2[count]
                    k = mutate_chromosome(k)
                    child.append(k)
                else:
                    k = mutate_chromosome(k)
                    child.append(k)

                count += 1
            pop_list.append(child)

    breed(children, fittest, second)

    # immigration to escape local minima
    rand1 = create_new_set(len(fittest))
    rand2 = create_new_set(len(fittest))

    pop_list.append(rand1)
    pop_list.append(rand2)

    breed(2, fittest, rand1)
    breed(2, fittest, rand2)

    # create mutants of fittest
    for i in range(mutants):
        mutant = []
        for k in fittest:
            b = random.random()
            if b < 0.2:
                mut_factor = 0.3
            else:
                mut_factor = 0.1
            k = k * np.random.normal(1, mut_factor)
            k = round(k, 4)
            mutant.append(k)
        pop_list.append(mutant)

    return pop_list

In [7]:

def test_against_real_data(patients: list, params: list):

    MDPE = []
    MDAPE = []
    wobble = []
    divergence = []

    for patient in patients:
        res = solve_for_custom(patient, params)
        
        MDPE.append(res["MDPE"])
        MDAPE.append(res["MDAPE"])
        wobble.append(res["Wobble"])
        divergence.append(res["Divergence"])
    

    data = (MDPE, MDAPE, wobble, divergence)
    return data

In [8]:
def single_core_test(job):

    results = test_against_real_data(job[0], job[1])
    
    MDPE = statistics.median(results[0])
    MDAPE = statistics.median(results[1])
    wobble = statistics.median(results[2])
    div = statistics.median(results[3])

    #calculate euclidian norm
    euclid_norm = np.linalg.norm([MDPE, MDAPE, wobble, div])
    
    result_dict = {
        "set": job[1],
        "results": {
            "Norm": abs(euclid_norm),
            "MDPE": MDPE,
            "MDAPE": MDAPE,
            "Wobble": wobble,
            "Divergence": div
            
        }
    }

    return result_dict

In [9]:
def compare_models(patients: list, model:str ='Marsh'):
    """
    Function to make comapring exisiting models easier.
    
    Valid options for model:
    - Marsh
    - Schnider
    - Eleveld
    
    All will test against all other valid options
    
    Returns:
    Dictionary of the type
    "model": model
    "results": {
        "norm": euclidian norm of other results
        "MDPE": median performance error
        "MDAPE": median absolute performance error
        "Wobble": wobble
    }
    """
    
    valid_models = ['Marsh', 'Schnider', 'Eleveld']
    
    if model not in valid_models:
        raise ValueError("Invalid model option")
        
    MDPE = []
    MDAPE = []
    wobble = []

    for patient in patients:
        
        if model == 'Marsh':
             patient_model = propofol.Marsh(patient["weight"])
        elif model == 'Schnider':
             patient_model = propofol.Schnider(patient["age"], patient["weight"], patient["height"], patient["sex"])
        elif model == 'Eleveld':
             patient_model = propofol.Eleveld(patient["age"], patient["weight"], patient["height"], patient["sex"])
        else:
            raise ValueError("Invalid model option")
            
        res =  solve_for_patient(patient_model, patient["events"])

        
        MDPE.append(res["MDPE"])
        MDAPE.append(res["MDAPE"])
        wobble.append(res["Wobble"])
        
    MDPE = statistics.median(MDPE)
    MDAPE = statistics.median(MDAPE)
    wobble = statistics.median(wobble)
    
    #calculate euclidian norm
    euclid_norm = np.linalg.norm([MDPE, MDAPE, wobble])
    
    result_dict = {
        "set": model,
        "results": {
            "Norm": abs(euclid_norm),
            "MDPE": MDPE,
            "MDAPE": MDAPE,
            "Wobble": wobble
            
        }
    }

    return result_dict

In [10]:
import time
from IPython import display
from multiprocessing import Pool
import traceback
import pprint
import math


print("creating population")
pop = create_new_population(10, 9)
pool = Pool()

creating population


In [11]:
def new_test_pop(pop):
    jobs = []
    for i in pop:
        jobs.append((Adults, i))

    result = pool.map(single_core_test, jobs)

    pop_list = []
    for i in result:
        if not math.isnan(i["results"]["Norm"]):
            pop_list.append((i["results"]["Norm"], i['set'], i["results"]))
    pop_list.sort()
#     output = (pop_list[0][0], pop_list[0][1], pop_list[1][0], pop_list[1][1])
    
    output = {
        "best_fitness": pop_list[0][2],
        "second_fitness":pop_list[1][2],
        "fittest_set": pop_list[0][1],
        "second_set": pop_list[1][1]
    }

    return output

In [12]:
res= new_test_pop(pop)

pprint.pprint(res)

{'best_fitness': {'Divergence': 0,
                  'MDAPE': 0.5463749632202608,
                  'MDPE': -0.4004613432363106,
                  'Norm': 0.7518401474157018,
                  'Wobble': 0.32614217667375167},
 'fittest_set': [0.5871,
                 0.7304,
                 0.3253,
                 0.8113,
                 0.1094,
                 0.0259,
                 0.3196,
                 0.0783,
                 0.3395],
 'second_fitness': {'Divergence': 0,
                    'MDAPE': 0.6609440901042103,
                    'MDPE': 0.05292675582138653,
                    'Norm': 0.8684046615330615,
                    'Wobble': 0.5607836699181709},
 'second_set': [0.9545,
                0.9602,
                0.54,
                0.0335,
                0.4517,
                0.7286,
                0.8705,
                0.2631,
                0.2279]}


In [13]:
# ####### REWRITE THIS ########

# print("beginning test")
# fit_results = new_test_pop(pop)
# fittest_set = fit_results["fittest_set"]
# second_set = fit_results["second_set"]

# max_tries = 5
# while fit_results["best_fitness"]["Norm"] > 9.9:
#     fit_results = test_population(pop)
#     best_fitness = fit_results["best_fitness"]
    
#     print ("trying again")    

# pltB = []
# pltS = []


# print("starting")
# # print(f'{"gen"}:3, {"Norm"}:10, {"MDAPE}":10, {"MDPE"}:10, {"Wobble"}:10, ')

# # 
# for i in range(25):
    
   
#     new_pop = mutate_population(10, fittest_set, second_set, 10)
#     fit_results = new_test_pop(new_pop) 
    
#     if fit_results["second_fitness"]["Norm"] < 99:
#         fittest_set = fit_results["fittest_set"]
#         best_fitness = fit_results["best_fitness"]["Norm"]
#         second_set = fit_results["second_set"]
#         second_fitness = fit_results["second_fitness"]["Norm"]

#         print(f'{i:3}, {round(fit_results["best_fitness"]["Norm"],5):10}, {round(fit_results["best_fitness"]["MDPE"],5):10}, {round(fit_results["best_fitness"]["MDAPE"],5):10}, {round(fit_results["best_fitness"]["Wobble"],5):10},')
        
# #         pltB.append(fit_results[1][1])
# #         pltS.append(fit_results[1][2])

# print(fittest_set)
# print ("finished!!")

In [14]:
fit_results = new_test_pop(pop)
fittest_set = fit_results["fittest_set"]
second_set = fit_results["second_set"]

max_tries = 5
while fit_results["best_fitness"]["Norm"] > 9.9:
    fit_results = test_population(pop)
    best_fitness = fit_results["best_fitness"]
    
    print ("trying again")    


In [15]:
def print_row(gen, results):
    print(f'{gen:5}|{results["Norm"]:10.5f}|{results["MDPE"]:10.5f}|{results["MDAPE"]:10.5f}|{results["Wobble"]:10.5f}')

print('-'*50)
print("beginning test")
print('-'*50)

start = time.time()

print('{:5}|{:>10}|{:>10}|{:>10}|{:>10}'.format('Gen','Norm', 'MDPE', 'MDAPE', 'Wobble'))
print('-'*50)


for i in range(20):
    
   
    new_pop = mutate_population(10, fittest_set, second_set, 10)
    fit_results = new_test_pop(new_pop) 
    
    fittest_set = fit_results["fittest_set"]
    best_fitness = fit_results["best_fitness"]["Norm"]
    second_set = fit_results["second_set"]
    second_fitness = fit_results["second_fitness"]["Norm"]

    print_row(i, fit_results["best_fitness"])
   
diff = time.time() - start
mins = diff/60

print('-'*50)
print(f'Run complete in {mins:.2f} minutes')
print('-'*50)


--------------------------------------------------
beginning test
--------------------------------------------------
Gen  |      Norm|      MDPE|     MDAPE|    Wobble
--------------------------------------------------
    0|   0.65185|   0.08175|   0.49606|   0.41492
    1|   0.59907|  -0.01555|   0.44026|   0.40597
    2|   0.55023|  -0.19156|   0.40803|   0.31555
    3|   0.44266|  -0.12458|   0.34108|   0.25316
    4|   0.40466|  -0.05926|   0.31586|   0.24591
    5|   0.38085|  -0.05208|   0.29375|   0.23674
    6|   0.36006|  -0.03915|   0.27894|   0.22428
    7|   0.34338|  -0.04114|   0.26456|   0.21500
    8|   0.33699|  -0.04132|   0.25951|   0.21096
    9|   0.33699|  -0.04132|   0.25951|   0.21096
   10|   0.33486|  -0.08606|   0.25945|   0.19342
   11|   0.33271|  -0.04582|   0.25785|   0.20521
   12|   0.32557|  -0.04128|   0.25384|   0.19964
   13|   0.32557|  -0.04128|   0.25384|   0.19964
   14|   0.32557|  -0.04128|   0.25384|   0.19964
   15|   0.32517|  -0.04931|   0

In [16]:
def test_custom(patients, params):
    
    MDPE = []
    MDAPE = []
    wobble = []

    for patient in patients:
        patient_model = HackDay_model(patient["age"], patient["weight"], patient["height"], patient["sex"], params )
        res =  solve_for_patient(patient_model, patient["events"])

        
        MDPE.append(res["MDPE"])
        MDAPE.append(res["MDAPE"])
        wobble.append(res["Wobble"])
        
    MDPE = statistics.median(MDPE)
    MDAPE = statistics.median(MDAPE)
    wobble = statistics.median(wobble)
    
    #calculate euclidian norm
    euclid_norm = np.linalg.norm([MDPE, MDAPE, wobble])
    
    result_dict = {
        "set": 'Custom',
        "results": {
            "Norm": abs(euclid_norm),
            "MDPE": MDPE,
            "MDAPE": MDAPE,
            "Wobble": wobble
            
        }
    }

    return result_dict

In [17]:
valid_models = ['Marsh', 'Schnider', 'Eleveld']

print('{:10}|{:>10}|{:>10}|{:>10}|{:>10}'.format('Model', 'Norm', 'MDPE', 'MDAPE', 'Wobble'))
print('-'*53)

for i in valid_models:
      result = compare_models(Adults, model= i)
      print(f'{i:10}|{result["results"]["Norm"]:10.5f}|{result["results"]["MDPE"]:10.5f}|{result["results"]["MDAPE"]:10.5f}|{result["results"]["Wobble"]:10.5f}')

    
i = 'Custom'
result = test_custom(Adults, fit_results["fittest_set"])
print(f'{i:10}|{result["results"]["Norm"]:10.5f}|{result["results"]["MDPE"]:10.5f}|{result["results"]["MDAPE"]:10.5f}|{result["results"]["Wobble"]:10.5f}')



Model     |      Norm|      MDPE|     MDAPE|    Wobble
-----------------------------------------------------
Marsh     |   0.44928|   0.13663|   0.34693|   0.25064
Schnider  |   0.56729|   0.13267|   0.37465|   0.40479
Eleveld   |   0.41478|   0.17150|   0.29664|   0.23374
Custom    |   0.32380|  -0.03719|   0.25433|   0.19692
